# Distracted driver detection
<br>這是一個Kaggle上面的題目：https://www.kaggle.com/c/state-farm-distracted-driver-detection</br>
<br>目的是為了預測圖片上的司機是否有在安全駕駛，圖片的資料原本是RGB，首先用python套件把資料轉換成Grayscale之後再利用PySpark分析</br>

In [1]:
%pylab inline
import os,cv2,glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
#在Notebook上把spark import進來
import sys

os.environ['SPARK_HOME']="C:/spark-1.6.1/" #看spark在你電腦的path是在哪裡

sys.path.append("C:/spark-1.6.1/bin") 
sys.path.append("C:/spark-1.6.1/python")
sys.path.append("C:/spark-1.6.1/python/pyspark/")
sys.path.append("C:/spark-1.6.1/python/lib") 
sys.path.append("C:/spark-1.6.1/python/lib/pyspark.zip")

import pyspark
from pyspark import SparkConf,SparkContext
conf = SparkConf().setAppName('driver_distraction_detect').setMaster('local')
sc = SparkContext(conf=conf)
sqlContext = pyspark.sql.SQLContext(sc) #DataFrame需要用到的SQL context

In [25]:
import pyspark.mllib
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.tree import DecisionTree

In [4]:
img_rows, img_cols = 96,128

In [5]:
def get_image(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    edges = cv2.Canny(resized, 90, 250)
    return edges

In [6]:
def load_train(img_rows, img_cols, color_type=1):
    X_train = []

    print('Read train images...')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('C:/Users/Jing Kai/Desktop/Ipython/Kaggle/Driver/sample/', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_image(fl, img_rows, img_cols, color_type)
            dims = shape(img)
            img_data= np.reshape(img, (1,dims[0] * dims[1]))[0]
            img_data = img_data.astype('float32') / 255.
            X_train.append(LabeledPoint(float(j), img_data))
            
    return X_train

In [7]:
X_train = load_train(img_rows,img_cols)

Read train images...
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9


In [14]:
hi = np.array([LabeledPoint(i,[i,j]) for i in range(10) for j in range(10)])

In [18]:
hi

array([LabeledPoint(0.0, [0.0,0.0]), LabeledPoint(0.0, [0.0,1.0]),
       LabeledPoint(0.0, [0.0,2.0]), LabeledPoint(0.0, [0.0,3.0]),
       LabeledPoint(0.0, [0.0,4.0]), LabeledPoint(0.0, [0.0,5.0]),
       LabeledPoint(0.0, [0.0,6.0]), LabeledPoint(0.0, [0.0,7.0]),
       LabeledPoint(0.0, [0.0,8.0]), LabeledPoint(0.0, [0.0,9.0]),
       LabeledPoint(1.0, [1.0,0.0]), LabeledPoint(1.0, [1.0,1.0]),
       LabeledPoint(1.0, [1.0,2.0]), LabeledPoint(1.0, [1.0,3.0]),
       LabeledPoint(1.0, [1.0,4.0]), LabeledPoint(1.0, [1.0,5.0]),
       LabeledPoint(1.0, [1.0,6.0]), LabeledPoint(1.0, [1.0,7.0]),
       LabeledPoint(1.0, [1.0,8.0]), LabeledPoint(1.0, [1.0,9.0]),
       LabeledPoint(2.0, [2.0,0.0]), LabeledPoint(2.0, [2.0,1.0]),
       LabeledPoint(2.0, [2.0,2.0]), LabeledPoint(2.0, [2.0,3.0]),
       LabeledPoint(2.0, [2.0,4.0]), LabeledPoint(2.0, [2.0,5.0]),
       LabeledPoint(2.0, [2.0,6.0]), LabeledPoint(2.0, [2.0,7.0]),
       LabeledPoint(2.0, [2.0,8.0]), LabeledPoint(2.0, [2.0,9.

In [22]:
data = [
...     LabeledPoint(0.0, [0.0]),
...     LabeledPoint(1.0, [1.0]),
...     LabeledPoint(1.0, [2.0]),
...     LabeledPoint(1.0, [3.0])
... ]

In [26]:
model = DecisionTree.trainClassifier(sc.parallelize(data), 2, {})

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-1.6.1\python\lib\pyspark.zip\pyspark\worker.py", line 64, in main
Exception: Python in worker has different version 2.7 than that in driver 3.5, PySpark cannot run with different minor versions

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:207)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:125)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.

In [15]:
len(hi)

100

In [16]:
data = sc.parallelize(hi)

In [17]:
data.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-1.6.1\python\lib\pyspark.zip\pyspark\worker.py", line 64, in main
Exception: Python in worker has different version 2.7 than that in driver 3.5, PySpark cannot run with different minor versions

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:207)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:125)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.

In [ ]:
training, valid = data.randomSplit([0.6, 0.4], seed=0)

In [ ]:
model = NaiveBayes.train(training)